# Corrigé — Exercices de Bac NSI — POO

In [1]:
# Vérification rapide de l'environnement Python
import sys, platform
print(sys.version)
print(platform.python_implementation(), platform.platform())

3.13.7 (main, Aug 14 2025, 00:00:00) [GCC 15.2.1 20250808 (Red Hat 15.2.1-1)]
CPython Linux-6.16.8-200.fc42.x86_64-x86_64-with-glibc2.41


## Exercice 1 — Classe simple : `Lampe`

**Objectifs :** attributs d'instance, méthode spéciale `__str__`, et **attribut de classe** pour compter les instances.

In [2]:
class Lampe:
    compteur = 0  # attribut de classe : nombre de lampes créées
    
    def __init__(self, couleur):
        self.couleur = couleur
        self.allumee = False  # valeur initiale
        Lampe.compteur += 1

    def allumer(self):
        self.allumee = True

    def eteindre(self):
        self.allumee = False

    def __str__(self):
        etat = "allumée" if self.allumee else "éteinte"
        return f"Lampe {self.couleur} ({etat})"

# Démonstration
l1 = Lampe("rouge")
l2 = Lampe("bleue")
l1.allumer()
print(l1)           # Lampe rouge (allumée)
print(l2)           # Lampe bleue (éteinte)
print(Lampe.compteur)  # 2

# Tests rapides
assert l2.allumee is False
assert Lampe.compteur >= 2

Lampe rouge (allumée)
Lampe bleue (éteinte)
2


## Exercice 2 — `Compte` bancaire avec `virement`

**Objectifs :** méthodes, encapsulation simple du solde, contrôle des préconditions (solde suffisant).

In [3]:
class Compte:
    def __init__(self, titulaire, solde=0):
        self.titulaire = titulaire
        self.solde = solde

    def deposer(self, montant):
        self.solde += montant

    def retirer(self, montant):
        if montant <= self.solde:
            self.solde -= montant
            return True
        return False

    def virement(self, autre_compte, montant):
        if self.retirer(montant):
            autre_compte.deposer(montant)
            return True
        return False

    def __str__(self):
        return f"Compte de {self.titulaire} : {self.solde} €"

# Démonstration
a = Compte("Alice", 100)
b = Compte("Bob", 20)
print(a, b)
assert a.virement(b, 50) is True
print(a, b)
assert a.virement(b, 80) is False  # solde insuffisant, pas de mouvement
print(a, b)

Compte de Alice : 100 € Compte de Bob : 20 €
Compte de Alice : 50 € Compte de Bob : 70 €
Compte de Alice : 50 € Compte de Bob : 70 €


## Exercice 3 — Héritage `Forme` → `Cercle`, `Rectangle`

**Objectifs :** héritage, redéfinition de méthode (`aire`), usage de `super()` et polymorphisme via une fonction.

In [4]:
import math

class Forme:
    def __init__(self, nom):
        self.nom = nom
    def aire(self):
        return 0

class Cercle(Forme):
    def __init__(self, rayon):
        super().__init__("cercle")
        self.rayon = rayon
    def aire(self):
        return math.pi * (self.rayon ** 2)

class Rectangle(Forme):
    def __init__(self, largeur, hauteur):
        super().__init__("rectangle")
        self.largeur = largeur
        self.hauteur = hauteur
    def aire(self):
        return self.largeur * self.hauteur

def afficher_aire(forme):
    print(f"Aire du {forme.nom} :", forme.aire())

# Démonstration
afficher_aire(Cercle(3))       # ~28.27
afficher_aire(Rectangle(4, 5)) # 20

# Tests rapides (tolérance)
assert abs(Cercle(3).aire() - 28.2743338823) < 1e-6
assert Rectangle(4,5).aire() == 20

Aire du cercle : 28.274333882308138
Aire du rectangle : 20


## Exercice 4 — Zoo : polymorphisme

**Objectifs :** héritage simple et redéfinition de méthode `parler()`.

In [5]:
class Animal:
    def __init__(self, nom):
        self.nom = nom
    def parler(self):
        return "..."

class Chien(Animal):
    def parler(self):
        return "Woof!"

class Chat(Animal):
    def parler(self):
        return "Miaou!"

animaux = [Chien("Rex"), Chat("Mina"), Animal("Mystère")]
for a in animaux:
    print(a.nom, "->", a.parler())

# Tests
assert Chien("X").parler() == "Woof!"
assert Chat("Y").parler() == "Miaou!"
assert Animal("Z").parler() == "..."

Rex -> Woof!
Mina -> Miaou!
Mystère -> ...


## Exercice 5 — Réseau social simplifié : `suggestions()`

**Objectifs :** relation bidirectionnelle, parcours des "amis de mes amis" et filtration (pas soi-même, pas déjà ami).

In [6]:
class Utilisateur:
    def __init__(self, nom):
        self.nom = nom
        self.amis = []

    def ajouter_ami(self, autre):
        if autre not in self.amis and autre != self:
            self.amis.append(autre)
            autre.amis.append(self)

    def est_ami(self, autre):
        return autre in self.amis

    def suggestions(self):
        s = set()
        for ami in self.amis:
            for candidat in ami.amis:
                if candidat is not self and candidat not in self.amis:
                    s.add(candidat)
        return list(s)

    def __str__(self):
        return f"{self.nom} ({len(self.amis)} amis)"

# Démonstration
alice = Utilisateur("Alice")
bob   = Utilisateur("Bob")
carla = Utilisateur("Carla")

alice.ajouter_ami(bob)
bob.ajouter_ami(carla)

print("Alice est amie avec Carla ?", alice.est_ami(carla))
print("Suggestions pour Alice :", [u.nom for u in alice.suggestions()])

# Tests
assert alice.est_ami(carla) is False
assert [u.nom for u in alice.suggestions()] == ["Carla"]

Alice est amie avec Carla ? False
Suggestions pour Alice : ['Carla']
